In [2]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [4]:
dir = 'pedestrian/input'

for f in os.listdir(dir):
    if f.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        image_path = os.path.join(dir, f)  # Full path to the image
        img = cv2.imread(image_path)
        cv2.imshow(dir, img)
        cv2.waitKey(1)
        
cv2.destroyAllWindows()

In [ ]:
step = 2

for i in range(300, 1100):
    if not i % step:
        I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
        cv2.imshow("I", I)
        cv2.waitKey(10)

cv2.destroyAllWindows()


### Odejmowanie

In [ ]:
def indexation_and_mark(dir, step, thresh):
    
    with open("%stemporalROI.txt" % dir, "r") as f:  # open file
        line = f.readline()  # read line
            
    roi_start, roi_end = line.split()  # split line
    roi_start = int(roi_start)  # conversion to int
    roi_end = int(roi_end)  # conversion to int
    
    prev_I = cv2.imread("%sinput/in%06d.jpg" % (dir, roi_start))
    prev_I = cv2.cvtColor(prev_I, cv2.COLOR_BGR2GRAY).astype(int)

    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i in range(roi_start + 1, roi_end):
        if not i % step:
            I = cv2.cvtColor(cv2.imread("%sinput/in%06d.jpg" % (dir, i)), cv2.COLOR_BGR2GRAY).astype(int)
            diff = np.abs(I - prev_I).astype(np.uint8)
            prev_I = I
            cv2.imshow("difference", diff)
            cv2.waitKey(10)
            B = (255 * (diff > thresh)).astype(np.uint8)
            # lub (T,  thresh) = cv2.threshold (D ,10 ,255 , cv2 . THRESH_BINARY)
            cv2.imshow("binarization", B)
            cv2.waitKey(10)
            # filtracja medianowa
            B = cv2.medianBlur(B, 3)
            kernel = np.ones((3, 3), np.uint8)
            # erozja i dylatacja po sobie - otwarcie morfologiczne
            Iopen = cv2.morphologyEx(B, cv2.MORPH_OPEN, kernel)
            cv2.imshow("opening", Iopen)
            cv2.waitKey(10)

            retval, labels, stats, centroids = cv2.connectedComponentsWithStats(B)
            # retval -- total number of unique labels
            # labels -- destination labeled image
            # stats -- statistics output for each label , including the background label .
            # centroids -- centroid output for each label , including the background label .

            cv2.imshow("Labels", np.uint8(labels / retval * 255))
            cv2.waitKey(10)

            I_VIS = cv2.imread("%sinput/in%06d.jpg" % (dir, i))
            if stats.shape[0] > 1:  # are there any objects
                tab = stats[1:, 4]  # 4 columns without first element
                pi = np.argmax(tab)  # finding the index of the largest item
                pi = pi + 1  # increment because we want the index in stats , not in tab
                # drawing a bbox
                cv2.rectangle(
                    I_VIS,
                    (stats[pi, 0], stats[pi, 1]),
                    (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]),
                    (255, 0, 0),
                    2,
                )
                # print information about the field and the number of the largest element
                cv2.putText(
                    I_VIS,
                    "%f" % stats[pi, 4],
                    (stats[pi, 0], stats[pi, 1]),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    (255, 0, 0),
                )
                cv2.putText(
                    I_VIS,
                    "%d" % pi,
                    (int(centroids[pi, 0]), int(centroids[pi, 1])),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    1,
                    (255, 0, 0),
                )

            cv2.imshow("indexation", I_VIS)
            cv2.waitKey(10)
            
            if i > roi_start and i < roi_end:
                gt_mask = cv2.imread("%sgroundtruth/gt%06d.png" % (dir, i), 0)   # 0 to szary obraz
                tp_m = np.logical_and((B == 255), (gt_mask == 255)) * 1  # logical product of the matrix elements
                tn_m = np.logical_and((B == 0), (gt_mask == 0)) * 1
                fp_m = np.logical_and((B == 255), (gt_mask == 0)) * 1
                fn_m = np.logical_and((B == 0), (gt_mask == 255)) * 1

                tp = np.sum(tp_m) + tp  # sum of the elements in the matrix and update sum
                tn = np.sum(tn_m) + tn
                fp = np.sum(fp_m) + fp
                fn = np.sum(fn_m) + fn

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = (2 * precision * recall) / (precision + recall)

    print(f"{dir[:-1]} precision: {precision}")
    print(f"{dir[:-1]} recall: {recall}")
    print(f"{dir[:-1]} f1: {f1}")

    cv2.destroyAllWindows()
    
    return precision, recall, f1


In [ ]:
pedestrian_out = indexation_and_mark("pedestrian/", step=1, thresh=20)
office_out = indexation_and_mark("office/", step=1, thresh=12)
highway_out = indexation_and_mark("highway/", step=1, thresh=20)

pedestrian precision: 0.7267733122323391
pedestrian recall: 0.4052502601131082
pedestrian f1: 0.5203514856412259
office precision: 0.9302603671699466
office recall: 0.03420698733431967
office f1: 0.06598752036300502
highway precision: 0.9446556905077627
highway recall: 0.4461302879738315
highway f1: 0.6060451022125748
